In [1]:
import torch # do not run after instantiating TorchClient
from grid.clients.torch import TorchClient
from keras.datasets import mnist
import numpy as np
import re

/Users/jasonmancuso/anaconda/envs/openmined/lib/python3.6/site-packages/h5py-2.7.1-py3.6-macosx-10.7-x86_64.egg/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
client = TorchClient(verbose=False)


UPDATE: Connecting to IPFS... this can take a few seconds...

SUCCESS: Connected!!! - My ID: client:QmeVrfEayAzvVM9Ujhyu4SjtQnNmQmmF7dvyRtMmzc9wh4
all parts .... ['', 'Users', 'jasonmancuso', '.openmined']
full path /
full path /Users/
full path /Users/jasonmancuso/
full path /Users/jasonmancuso/.openmined/

UPDATE: Querying known workers...
	WORKER: /p2p-circuit/ipfs/QmXVoAQN2hHf5VF5SfCV4kQqZqNuZi3PE1so3pMYFguSUS...SUCCESS!!!
	WORKER: /p2p-circuit/ipfs/QmQabt3SWuDvjse9z7GAcH2BGQv4wH8bumkd4x5oXN2obX...SUCCESS!!!
	WORKER: /p2p-circuit/ipfs/QmXkWUybbTnfvFH8SUcrug6RGTLYTB23gSockKLxueR1vQ...SUCCESS!!!


UPDATE: Searching for IPFS nodes - 16 found overall - 5 are OpenMined workers          

SUCCESS: Found 5 OpenMined nodes!!!

Hooking into Torch...
Overloading complete.


In [3]:
# Identify compute nodes
compute_nodes = [x for x in client if re.match('compute:', x)]
for x in compute_nodes:
    print(x)

compute:QmPWXB3APBhPJUZ2SDQhRxWpfFxsRSoC99roBivYKZAtTq
compute:QmeVrfEayAzvVM9Ujhyu4SjtQnNmQmmF7dvyRtMmzc9wh4


In [4]:
laptop = compute_nodes[0]
colab = compute_nodes[1]

In [5]:
assign_tensors_to_nodes = {}
assign_tensors_to_nodes[colab] = []
assign_tensors_to_nodes[laptop] = []

In [6]:
# Loop over available nodes
for node in compute_nodes:
    for i in range(5):
        # Just getting five randomly generated 2x2 matrices here
        assign_tensors_to_nodes[node].append(torch.FloatTensor(3,3))
        
# Fill desktop tensors with 1s
for tensor in assign_tensors_to_nodes[colab]:
    tensor.fill_(1)
    
# Fill desktop tensors with 2s
for tensor in assign_tensors_to_nodes[laptop]:
    tensor.fill_(2)

In [7]:
assign_tensors_to_nodes[laptop]

[
  2  2  2
  2  2  2
  2  2  2
 [torch.FloatTensor of size 3x3], 
  2  2  2
  2  2  2
  2  2  2
 [torch.FloatTensor of size 3x3], 
  2  2  2
  2  2  2
  2  2  2
 [torch.FloatTensor of size 3x3], 
  2  2  2
  2  2  2
  2  2  2
 [torch.FloatTensor of size 3x3], 
  2  2  2
  2  2  2
  2  2  2
 [torch.FloatTensor of size 3x3]]

### Barebones MNIST on two workers

Processing.

In [8]:
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [9]:
x_train = torch.FloatTensor(x_train).view(len(x_train),-1) / 255

y_train = torch.LongTensor(y_train.astype('int64').tolist())

In [10]:
def subset_tensors(tens, labels, p=.1):
    """Given a tensor and labels, get a subset of proportion `p`"""
    sub_mask = torch.FloatTensor(len(tens)).uniform_() < p
    tens = torch.masked_select(tens, sub_mask.unsqueeze(1).expand(-1,784)).view(sum(sub_mask),-1)
    labels = torch.masked_select(labels, sub_mask).view(sum(sub_mask),-1)
    return tens, labels

In [11]:
x, y = subset_tensors(x_train, y_train, p=.005)

Hyperparams

In [12]:
batch_size = 16
step_size = .001
epochs = 5

input_shape = x.shape[1]
first_neurons = 128
second_neurons = 64
n_classes = y.max() + 1

Split into batches and send to workers.

In [13]:
batched_x = torch.split(x, batch_size)
batched_y = torch.split(y, batch_size)

In [14]:
assert len(batched_x) == len(batched_y)
print(len(batched_x))

19


In [15]:
%%time

for (ix, (x_i, y_i)) in enumerate(zip(batched_x, batched_y)):
    if ix == len(batched_x) - 2:
        break
    x_i.send_(compute_nodes[ix % 2])
    y_i.send_(compute_nodes[ix % 2])

CPU times: user 433 ms, sys: 20.2 ms, total: 453 ms
Wall time: 479 ms


Build model.

In [16]:
# TODO: initialize properly, or stay lazy
W_0 = torch.FloatTensor(input_shape, first_neurons).uniform_()*.05
W_1 = torch.FloatTensor(first_neurons, second_neurons).uniform_()*.05
W_2 = torch.FloatTensor(second_neurons, n_classes).uniform_()*.05

In [17]:
def relu(x, d=False):
    z = x.clone()
    z[z<=0] = 0
    if not d:
        return z
    else:
        z[x>0] = 1
        return z

def linear(x, weights, d=False):
    if not d:
        return x.matmul(weights)
    else:
        return x

def softmax(x):
    exps = torch.exp(x - torch.max(x))
    return exps / torch.sum(exps, -1, keepdim=True)

def cross_entropy(x, y, d=False):
    batch = x.size(0)
    p = softmax(x)
    if not d:
        log_likelihood = -torch.log(p[range(batch), y])
        loss = torch.sum(torch.sum(log_likelihood, 1), 0,keepdim=True) / batch
        return loss
    else:
        p[range(batch),y] -= 1
        return p / batch

In [18]:
# Initialize grad holders
W_0_grads = []
W_1_grads = []
W_2_grads = []

In [20]:
W_2.sum()

15.99358927020512

In [24]:
#for e in range(epochs):
for (ix, (x_i, y_i)) in enumerate(zip(batched_x, batched_y)):
    # Return clones of new weights to workers
    # TODO: Debug ConnectionError (need to sleep)
    W_0_clones = [W_0.clone().send_(node) for node in compute_nodes]
    W_1_clones = [W_1.clone().send_(node) for node in compute_nodes]
    W_2_clones = [W_2.clone().send_(node) for node in compute_nodes]

    # Pull pointers from list of clones
    W_0_tmp = W_0_clones[ix % 2]
    W_1_tmp = W_1_clones[ix % 2]
    W_2_tmp = W_2_clones[ix % 2]

    # Forward pass
    act_0 = relu(linear(x_i, W_0_tmp))
    act_1 = relu(linear(act_0, W_1_tmp))
    logits = linear(act_1, W_2_tmp)

    # Calculate gradient components
    # TODO: Double check math
    #  it's super late and I haven't done this in forever
    out_grad = cross_entropy(logits, y_i, d=True)
    logits_grad = act_1
    act_1_grad = relu(act_1, d=True)
    act_0_grad = relu(act_0, d=True)

    # Calculate gradient with respect to weights
    # TODO: Fill this in...
    #  it's super late and I haven't done this in forever
    W_2_grad = out_grad.t().matmul(logits_grad)
    #W_1_grad = W_2_grad * ...
    #W_0_grad = ...

    # Recall gradient
    #W_0_grads[ix % 2] = W_0_grad.get_()
    #W_1_grads[ix % 2] = W_1_grad.get_()
    W_2_grads[ix % 2] = W_2_grad.get_()

    # this version of distributed sgd is simple and arbitrary
    # can extend this to do k-step syncing and asynchronous updates
    # with a few tweaks
    if ix % 2 == 1:
        # Average gradients
        #W_0_grad = sum(W_0_grads) / len(W_0_grads)
        #W_1_grad = sum(W_1_grads) / len(W_1_grads)
        W_2_grad = sum(W_2_grads) / len(W_2_grads)

        # Take a step in weight space from master weights
        #W_0 -= step_size * W_0_grad
        #W_1 -= step_size * W_1_grad
        W_2 -= step_size * W_2_grad

ConnectionError: ConnectionError: ('Connection aborted.', BrokenPipeError(32, 'Broken pipe'))

In [22]:
W_2.sum()

15.99358927020512